In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, cross_val_score, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from datetime import datetime

In [2]:
df = pd.read_csv("spotify_reviews.csv")

In [3]:
df.shape

(40000, 6)

In [4]:
df.columns

Index(['Review', 'Rating', 'sentiment', 'review_after_removing_html_tags',
       'review_after_removing_punctuation',
       'review_after_pos_and_lemmatization'],
      dtype='object')

In [5]:
df.isna().sum()

Review                                0
Rating                                0
sentiment                             0
review_after_removing_html_tags       0
review_after_removing_punctuation     0
review_after_pos_and_lemmatization    0
dtype: int64

In [6]:
df[ df.sentiment == 1 ].shape

(20000, 6)

In [7]:
df[ df.sentiment == 0 ].shape

(20000, 6)

In [9]:
bag_of_word_model = CountVectorizer()

X = bag_of_word_model.fit_transform(df["review_after_pos_and_lemmatization"])
y = df["sentiment"]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
y_train[ y_train == 1 ].shape

(15945,)

In [12]:
y_train[ y_train == 0 ].shape

(16055,)

In [13]:
y_test[ y_test == 1 ].shape

(4055,)

In [14]:
y_test[ y_test == 0 ].shape

(3945,)

#### KFold cross-validation

In [15]:
# Set up k-fold cross-validation
kf = KFold(n_splits=4, shuffle=True)

random_forest_scores = []
logistic_scores = []
svm_scores = []
gd_boost_scores = []

# Generate indices for each fold
for train_index, test_index in kf.split(X):
    x_train_data, x_test_data = X[train_index], X[test_index]
    y_train_data, y_test_data = y[train_index], y[test_index]

    ## Random Forest
    model_forest = RandomForestClassifier(max_depth=10, n_estimators=500)

    print("Fitting random forest to training data....")    
    model_forest = model_forest.fit(x_train_data, y_train_data)
    
    y_pred_data = model_forest.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    random_forest_scores.append(score)
    print(random_forest_scores)
    
    ## Logistic Regression
    
    model_logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', C=0.1)
    print("Fitting Logistic Regression to training data....")  
    model_logistic_regression = model_logistic_regression.fit(x_train_data, y_train_data)
    
    y_pred_data = model_logistic_regression.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    logistic_scores.append(score)
    print(logistic_scores)
    
    ## SVM
    model_svm = SVC(C=0.1, kernel='linear')
    print("Fitting SVM to training data....")  
    model_svm = model_svm.fit(x_train_data, y_train_data)
    
    y_pred_data = model_svm.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    svm_scores.append(score)
    print(svm_scores)
    
    ## Gradient Boosting
    gb_classifier = GradientBoostingClassifier(random_state=2)
    print("Fitting Gradient Boosting to training data....")    
    gb_classifier.fit(x_train_data, y_train_data)
    
    y_pred_data = gb_classifier.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    gd_boost_scores.append(score)
    print(gd_boost_scores)
    
    print("=======================================================")

# print(f"Cross-validation scores: {scores}")
# print(f"Average score: {np.mean(scores)}")

Fitting random forest to training data....
[0.7924]
Fitting Logistic Regression to training data....
[0.8351]
Fitting SVM to training data....
[0.8473]
Fitting Gradient Boosting to training data....
[0.8071]
Fitting random forest to training data....
[0.7924, 0.7896]
Fitting Logistic Regression to training data....
[0.8351, 0.8365]
Fitting SVM to training data....
[0.8473, 0.8454]
Fitting Gradient Boosting to training data....
[0.8071, 0.8059]
Fitting random forest to training data....
[0.7924, 0.7896, 0.7894]
Fitting Logistic Regression to training data....
[0.8351, 0.8365, 0.8417]
Fitting SVM to training data....
[0.8473, 0.8454, 0.8522]
Fitting Gradient Boosting to training data....
[0.8071, 0.8059, 0.8111]
Fitting random forest to training data....
[0.7924, 0.7896, 0.7894, 0.8015]
Fitting Logistic Regression to training data....
[0.8351, 0.8365, 0.8417, 0.8397]
Fitting SVM to training data....
[0.8473, 0.8454, 0.8522, 0.8526]
Fitting Gradient Boosting to training data....
[0.8071, 

In [16]:
print("Random Forest KFold cross-validation")
print(f"Cross-validation scores: {random_forest_scores}")
print(f"Average score: {np.mean(random_forest_scores)}\n")
print("Logistic Regression KFold cross-validation")
print(f"Cross-validation scores: {logistic_scores}")
print(f"Average score: {np.mean(logistic_scores)}\n")
print("SVM cross-validation")
print(f"Cross-validation scores: {svm_scores}")
print(f"Average score: {np.mean(svm_scores)}\n")
print("Gradient Boosting KFold cross-validation")
print(f"Cross-validation scores: {gd_boost_scores}")
print(f"Average score: {np.mean(gd_boost_scores)}\n")

Random Forest KFold cross-validation
Cross-validation scores: [0.7924, 0.7896, 0.7894, 0.8015]
Average score: 0.793225

Logistic Regression KFold cross-validation
Cross-validation scores: [0.8351, 0.8365, 0.8417, 0.8397]
Average score: 0.83825

SVM cross-validation
Cross-validation scores: [0.8473, 0.8454, 0.8522, 0.8526]
Average score: 0.849375

Gradient Boosting KFold cross-validation
Cross-validation scores: [0.8071, 0.8059, 0.8111, 0.8078]
Average score: 0.807975



#### Random Forest

In [17]:
# Start the timer
start_time = datetime.now()

model_forest = RandomForestClassifier(max_depth=10, n_estimators=500)

print("Fitting random forest to training data....")    
model_forest = model_forest.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Random Forest Training Runtime:', runtime)

Fitting random forest to training data....
Random Forest Training Runtime: 0:00:23.453126


In [18]:
model_forest.score(X_test, y_test)

0.793625

In [19]:
model_forest.score(X_train, y_train)

0.81203125

In [20]:
# Start the timer
start_time = datetime.now()

y_pred = model_forest.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Random Forest Prediction Runtime:', runtime)

Random Forest Prediction Runtime: 0:00:00.375163


In [21]:
y_pred = cross_val_predict(model_forest, X_test, y_test)

In [22]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.71      0.76      3945
           1       0.75      0.84      0.79      4055

    accuracy                           0.78      8000
   macro avg       0.78      0.78      0.78      8000
weighted avg       0.78      0.78      0.78      8000



In [23]:
print(confusion_matrix(y_test, y_pred))

[[2817 1128]
 [ 650 3405]]


In [24]:
filename = 'models/rd_ngram_ml_model.sav'
pickle.dump(model_forest, open(filename, 'wb'))

#### Logistic Regression

In [25]:
# Start the timer
start_time = datetime.now()

model_logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', C=0.1)

print("Fitting Logistic Regression to training data....")  
model_logistic_regression = model_logistic_regression.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Logistic Regression Training Runtime:', runtime)

Fitting Logistic Regression to training data....
Logistic Regression Training Runtime: 0:00:00.108962


In [26]:
model_logistic_regression.score(X_test, y_test)

0.834

In [27]:
model_logistic_regression.score(X_train, y_train)

0.84534375

In [28]:
# Start the timer
start_time = datetime.now()

y_pred = model_logistic_regression.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Logistic Regression Prediction Runtime:', runtime)

Logistic Regression Prediction Runtime: 0:00:00.000968


In [29]:
y_pred = cross_val_predict(model_logistic_regression, X_test, y_test)

In [30]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.82      0.81      3945
           1       0.82      0.80      0.81      4055

    accuracy                           0.81      8000
   macro avg       0.81      0.81      0.81      8000
weighted avg       0.81      0.81      0.81      8000



In [31]:
print(confusion_matrix(y_test, y_pred))

[[3220  725]
 [ 821 3234]]


In [32]:
filename = 'models/lg_ngram_ml_model.sav'
pickle.dump(model_logistic_regression, open(filename, 'wb'))

#### SVM

In [33]:
# Start the timer
start_time = datetime.now()

model_svm = SVC(C=0.1, kernel='linear')

model_svm = model_svm.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:02:06.726303


In [34]:
model_svm.score(X_test, y_test)

0.84475

In [35]:
model_svm.score(X_train, y_train)

0.88425

In [36]:
# Start the timer
start_time = datetime.now()

y_pred = model_svm.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:00:12.704472


In [37]:
y_pred = cross_val_predict(model_svm, X_test, y_test)

In [38]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.83      0.83      3945
           1       0.83      0.82      0.83      4055

    accuracy                           0.83      8000
   macro avg       0.83      0.83      0.83      8000
weighted avg       0.83      0.83      0.83      8000



In [39]:
print(confusion_matrix(y_test, y_pred))

[[3280  665]
 [ 713 3342]]


In [40]:
filename = 'models/svm_ngram_ml_model.sav'
pickle.dump(model_svm, open(filename, 'wb'))

#### Gradient Boosting Classifier

In [41]:
# Start the timer
start_time = datetime.now()


# create a Gradient Boosting classifier with default hyperparameters
gb_classifier = GradientBoostingClassifier(random_state=2)

# fit the classifier to the training data
gb_classifier.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Gradient Boosting Prediction Runtime:', runtime)


Gradient Boosting Prediction Runtime: 0:01:00.695849


In [42]:
gb_classifier.score(X_test, y_test)

0.801875

In [43]:
gb_classifier.score(X_train, y_train)

0.81675

In [44]:
# Start the timer
start_time = datetime.now()

y_pred = gb_classifier.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:00:00.022233


In [45]:
y_pred = cross_val_predict(gb_classifier, X_test, y_test)

In [46]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.80      0.79      3945
           1       0.80      0.80      0.80      4055

    accuracy                           0.80      8000
   macro avg       0.80      0.80      0.80      8000
weighted avg       0.80      0.80      0.80      8000



In [47]:
filename = 'models/gb_boosting_ngram_ml_model.sav'
pickle.dump(gb_classifier, open(filename, 'wb'))